In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tqdm as tqdm

import matplotlib.pyplot as plt

%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn


# special matplotlib argument for improved plots
from matplotlib import rcParams

In [ ]:
data = pd.read_csv("MSN Historical Data.csv")
data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2017/12/1,68500,67000,69400,66500,725370,0.02
1,2017/12/4,69000,70000,70000,68800,1190000,0.01
2,2017/12/5,68900,69000,70100,68600,777070,0.00
3,2017/12/6,67700,68900,69000,66000,1080000,-0.02
4,2017/12/7,69000,68600,69800,67300,1560000,0.02


In [ ]:
dataset = data.loc[:,["Date","Price"]]
dataset.head()

,Date,Price
0,2017/12/1,68500
1,2017/12/4,69000
2,2017/12/5,68900
3,2017/12/6,67700
4,2017/12/7,69000


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1249 non-null   object
 1   Price   1249 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 19.6+ KB


In [ ]:
dataset.Date = pd.to_datetime(dataset.Date,format = "%Y/%m/%d")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1249 non-null   datetime64[ns]
 1   Price   1249 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 19.6 KB


In [ ]:
dataset = dataset.rename(columns = {"Price" : "y"})
dataset = dataset.rename(columns = {"Date" : "ds"})
dataset.head()

,ds,y
0,2017-12-01,68500
1,2017-12-04,69000
2,2017-12-05,68900
3,2017-12-06,67700
4,2017-12-07,69000


In [ ]:
test_days = 31
training_set = dataset.iloc[:-test_days, :]
test_set = dataset.iloc[-test_days:, :]
test_set.tail(5)

,ds,y
1244,2022-11-25,94000
1245,2022-11-28,97200
1246,2022-11-29,99000
1247,2022-11-30,102000
1248,2022-12-01,99000


In [ ]:

print(len(training_set))
print(len(test_set))

1218
31


In [ ]:
training_set.tail(5)

,ds,y
1213,2022-10-13,78700
1214,2022-10-14,79000
1215,2022-10-17,79000
1216,2022-10-18,82000
1217,2022-10-19,80000


In [ ]:
m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = 10,
            holidays_prior_scale = 10,
            changepoint_prior_scale = 0.05)
m.fit(training_set)

In [ ]:
future = m.make_future_dataframe(periods = len(test_set), freq = "D")
future.tail()

,ds
1244,2022-11-15
1245,2022-11-16
1246,2022-11-17
1247,2022-11-18
1248,2022-11-19


In [ ]:
future = pd.concat([future, dataset.iloc[:,1:]],
                   axis = 1)
future.tail(5)

,ds,y
1244,2022-11-15,94000
1245,2022-11-16,97200
1246,2022-11-17,99000
1247,2022-11-18,102000
1248,2022-11-19,99000


In [ ]:
forecast = m.predict(future)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2017-12-01,6807.607189,62904.591063,73103.471595,6807.607189,6807.607189,8.982570,8.982570,8.982570,9.261507,9.261507,9.261507,-0.278936,-0.278936,-0.278936,0.0,0.0,0.0,67957.417413
1,2017-12-04,6888.973039,63402.011050,74204.587762,6888.973039,6888.973039,9.016582,9.016582,9.016582,9.241034,9.241034,9.241034,-0.224452,-0.224452,-0.224452,0.0,0.0,0.0,69003.961779
2,2017-12-05,6916.094989,64327.283367,74784.382588,6916.094989,6916.094989,9.053752,9.053752,9.053752,9.255716,9.255716,9.255716,-0.201965,-0.201965,-0.201965,0.0,0.0,0.0,69532.700806
3,2017-12-06,6943.216939,64917.858534,75143.066095,6943.216939,6943.216939,9.084630,9.084630,9.084630,9.262316,9.262316,9.262316,-0.177686,-0.177686,-0.177686,0.0,0.0,0.0,70019.772248
4,2017-12-07,6970.338889,65268.454975,75826.445911,6970.338889,6970.338889,9.117084,9.117084,9.117084,9.268959,9.268959,9.268959,-0.151874,-0.151874,-0.151874,0.0,0.0,0.0,70519.505548


In [ ]:
predictions_prophet = forecast.yhat[-test_days:].rename("prophet")
predictions_prophet[:5]

1218     90544.296294
1219     90599.137226
1220   -224695.626918
1221   -224190.869135
1222     91063.008373
Name: prophet, dtype: float64

In [ ]:
print(len(test_set['y']))
print(len(predictions_prophet))

31
31


In [ ]:
#MAE and RMSE
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(round(mean_absolute_error(test_set['y'], predictions_prophet),0))
print(round(np.sqrt(mean_squared_error(test_set['y'], predictions_prophet)), 0))

93099.0
163949.0


In [ ]:
#MAPE function
def MAPE(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
MAPE(test_set['y'], predictions_prophet)

109.24336391603183

In [ ]:
from fbprophet.diagnostics import cross_validation
cs = cross_validation(m,
                         #Dự đoán 30 ngày tiếp theo
                         horizon = '30 days',
                         #Lấy 540 ngày dể đào tạo
                         initial = '540 days',
                         #Xử lý song song
                         parallel = "processes")

INFO:fbprophet:Making 81 forecasts with cutoffs between 2019-06-07 00:00:00 and 2022-09-19 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fcb06865a90>


In [ ]:
print(round(mean_absolute_error(cs['y'], cs['yhat']),0))
print(round(np.sqrt(mean_squared_error(cs['y'], cs['yhat'])), 0))
print(MAPE(cs['y'], cs['yhat']))

7745.0
10467.0
9.72158783312241


In [ ]:
y_true = cs['y'].values
y_pre = cs['yhat'].values

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
mae = mean_absolute_error(y_true,y_pre)
mape = mean_absolute_percentage_error(y_true,y_pre)
mse = mean_squared_error(y_true, y_pre)
rmse = np.sqrt(mse)
r2 = r2_score(y_true,y_pre)

print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape * 100:.2f}%")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2: {r2:.2f}")

MAE: 7745.25
MAPE: 9.72%
MSE: 109550918.84
RMSE: 10466.66
R2: 0.86


In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {'seasonality_prior_scale': np.arange(5, 10, 5), #[5, 10]  np.arange(5, 20, 5),
              'changepoint_prior_scale': np.arange(0.01, 0.05, 0.01),  # [0.01, 0.02, 0.03, 0.04, 0.05]
              'holidays_prior_scale': np.arange(5, 10, 5)}#[5, 10]
grid = ParameterGrid(param_grid)
list(grid)[:3]  

[{'changepoint_prior_scale': 0.01,
  'holidays_prior_scale': 5,
  'seasonality_prior_scale': 5},
 {'changepoint_prior_scale': 0.02,
  'holidays_prior_scale': 5,
  'seasonality_prior_scale': 5},
 {'changepoint_prior_scale': 0.03,
  'holidays_prior_scale': 5,
  'seasonality_prior_scale': 5}]

In [ ]:
rmse = []
for params in grid:
  #build model
  m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = params['seasonality_prior_scale'],
            holidays_prior_scale = params['holidays_prior_scale'],
            changepoint_prior_scale = params['changepoint_prior_scale'])
  m.fit(training_set)

  #cross-validation
  cs = cross_validation(m,
                         horizon = '30 days',
                         initial = '540 days',
                         parallel = "processes")
  
  #gather the results
  error = np.sqrt(mean_squared_error(cs['y'], 
                                     cs['yhat']))
  rmse.append(error)

INFO:fbprophet:Making 81 forecasts with cutoffs between 2019-06-07 00:00:00 and 2022-09-19 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fcb203c64f0>
INFO:fbprophet:Making 81 forecasts with cutoffs between 2019-06-07 00:00:00 and 2022-09-19 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fcb06806220>
INFO:fbprophet:Making 81 forecasts with cutoffs between 2019-06-07 00:00:00 and 2022-09-19 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fcb068deeb0>
INFO:fbprophet:Making 81 forecasts with cutoffs between 2019-06-07 00:00:00 and 2022-09-19 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7fcb06800a30>


In [ ]:
  best_params = grid[np.argmin(rmse)]
print(best_params)

In [ ]:
m = Prophet(growth = "linear",
          yearly_seasonality = True,
          weekly_seasonality = True,
          daily_seasonality = False,
          seasonality_mode = "multiplicative",
          seasonality_prior_scale = best_params['seasonality_prior_scale'],
          holidays_prior_scale = best_params['holidays_prior_scale'],
          changepoint_prior_scale = best_params['changepoint_prior_scale'])
m.fit(training_set)

In [ ]:
future = m.make_future_dataframe(periods = len(test_set),
                                 freq = "D")
future = pd.concat([future, dataset.iloc[:,1:]],
                   axis = 1)
future.tail(1)

In [ ]:
forecast = m.predict(future)
forecast.head()

In [ ]:
print(len(forecast['yhat']))
print(len(forecast.yhat[-test_set:]))
print(len(test_set["ds"]))

In [ ]:
predictions_prophet_tuned = forecast.yhat[-test_days:].rename("prophet_tuned")
predictions_prophet_tuned.index = test_set["ds"]
predictions_prophet_tuned.head()